In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/9 covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [2]:
df['city'].value_counts()

,count
city,
Kolkata,32
Bangalore,30
Delhi,22
Mumbai,16


Train Test Split

In [3]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['has_covid'])
y = df['has_covid']

X_train,X_test,y_train,y_test = train_test_split(X,y,
                                                test_size=0.2)

In [4]:
X_train.shape,y_train.shape,y_test.shape,X_test.shape

((80, 5), (80,), (20,), (20, 5))

1 Without Column Transformer

In [5]:
# missing values fill
from sklearn.impute import SimpleImputer
si = SimpleImputer()

X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])
X_train_fever.shape

(80, 1)

In [6]:
# encoding categorical values
from sklearn.preprocessing import OrdinalEncoder
Ordinal = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = Ordinal.fit_transform(X_train[['cough']])
X_test_cough = Ordinal.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [8]:
from sklearn.preprocessing import OneHotEncoder
# Replace 'sparse' with 'sparse_output'
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [9]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [10]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

2 With Column Transformer

In [11]:
from sklearn.compose import ColumnTransformer

In [14]:
from sklearn.compose import ColumnTransformer

tf = ColumnTransformer(
    transformers=[
        ("tf1", SimpleImputer(), ['fever']),
        ("tf2", OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
        # Replacing 'sparse' with 'sparse_output' to align with newer versions of scikit-learn
        ("tf3", OneHotEncoder(drop='first', sparse_output=False), ['gender', 'city']),
    ],
    remainder="passthrough"
)

In [15]:
tf

ColumnTransformer(remainder='passthrough',
                  transformers=[('tf1', SimpleImputer(), ['fever']),
                                ('tf2',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('tf3',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['gender', 'city'])])

In [16]:
X_train_tf = tf.fit_transform(X_train)
X_test_tf = tf.fit_transform(X_test)

In [17]:
X_train_tf

array([[ 99.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  66.        ],
       [100.87671233,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [101.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  49.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  54.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  12.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  34.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  75.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.    